In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
from torch import optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Function, Variable
from pathlib import Path
from itertools import groupby

%load_ext autoreload
%autoreload 2

In [10]:
from src.U_Net import UNet
from src.generator import train_generator, test_generator

from src.utils import run_length

In [3]:
input_dir = "./data/"
train_img_dir = "./data/train/"
test_img_dir = "./data/test/"

WIDTH = 256
HEIGHT = 256
category_num = 46 + 1

ratio = 8

epoch_num = 1
batch_size = 4

device = "cuda:0"

In [4]:
# len(os.listdir("./data/train/"))

In [5]:
train_df = pd.read_csv(input_dir + "train.csv")
# train_df.head()

,ImageId,EncodedPixels,Height,Width,ClassId
0,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6
1,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0
2,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,8521389 10 8526585 30 8531789 42 8537002 46 85...,5214,3676,28
3,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,12903854 2 12909064 7 12914275 10 12919485 15 ...,5214,3676,31
4,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,10837337 5 10842542 14 10847746 24 10852951 33...,5214,3676,32


# Lets Train

In [6]:
net = UNet(n_channels=3, n_classes=category_num).to(device)

optimizer = optim.SGD(
    net.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=0.0005
)

criterion = nn.CrossEntropyLoss()

In [7]:
val_sta = 73352
val_end = 83351
train_loss = []
valid_loss = []
for epoch in range(epoch_num):
    epoch_trn_loss = 0
    train_len = 0
    net.train()
    for iteration, (X_trn, Y_trn) in enumerate(tqdm(train_generator(train_df.iloc[:val_sta, :], batch_size,category_num,
                                                                    WIDTH,HEIGHT))):
        X = torch.tensor(X_trn, dtype=torch.float32).to(device)
        Y = torch.tensor(Y_trn, dtype=torch.long).to(device)
        train_len += len(X)
        
        #Y_flat = Y.view(-1)
        mask_pred = net(X)
        #mask_prob = torch.softmax(mask_pred, dim=1)
        #mask_prob_flat = mask_prob.view(-1)
        loss = criterion(mask_pred, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_trn_loss += loss.item()
        
        if iteration % 100 == 0:
            print("train loss in {:0>2}epoch  /{:>5}iter:    {:<10.8}".format(epoch+1, iteration, epoch_trn_loss/(iteration+1)))
        
    train_loss.append(epoch_trn_loss/(iteration+1))
    print("train {}epoch loss({}iteration):    {:10.8}".format(epoch+1, iteration, train_loss[-1]))
    
    epoch_val_loss = 0
    val_len = 0
    net.eval()
    for iteration, (X_val, Y_val) in enumerate(tqdm(train_generator(train_df.iloc[val_sta:val_end, :], batch_size,category_num,
                                                                    WIDTH,HEIGHT))):
        X = torch.tensor(X_val, dtype=torch.float32).to(device)
        Y = torch.tensor(Y_val, dtype=torch.long).to(device)
        val_len += len(X)
        
        #Y_flat = Y.view(-1)
        
        mask_pred = net(X)
        #mask_prob = torch.softmax(mask_pred, dim=1)
        #mask_prob_flat = mask_prob.view(-1)
        loss = criterion(mask_pred, Y)
        epoch_val_loss += loss.item()
        
        if iteration % 100 == 0:
            print("valid loss in {:0>2}epoch  /{:>5}iter:    {:<10.8}".format(epoch+1, iteration, epoch_val_loss/(iteration+1)))
        
    valid_loss.append(epoch_val_loss/(iteration+1))
    print("valid {}epoch loss({}iteration):    {:10.8}".format(epoch+1, iteration, valid_loss[-1]))

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


train loss in 01epoch  /    0iter:    3.841593  
train loss in 01epoch  /  100iter:    1.3038623 
train loss in 01epoch  /  200iter:    1.1614525 
train loss in 01epoch  /  300iter:    1.128743  
train loss in 01epoch  /  400iter:    1.1072618 
train loss in 01epoch  /  500iter:    1.0880041 
train loss in 01epoch  /  600iter:    1.082367  
train loss in 01epoch  /  700iter:    1.0788356 
train loss in 01epoch  /  800iter:    1.0730438 
train loss in 01epoch  /  900iter:    1.0719689 
train loss in 01epoch  / 1000iter:    1.0646555 
train loss in 01epoch  / 1100iter:    1.0606396 
train loss in 01epoch  / 1200iter:    1.0542481 
train loss in 01epoch  / 1300iter:    1.0500518 
train loss in 01epoch  / 1400iter:    1.0455208 
train loss in 01epoch  / 1500iter:    1.0376431 
train loss in 01epoch  / 1600iter:    1.0302224 
train loss in 01epoch  / 1700iter:    1.0247685 
train loss in 01epoch  / 1800iter:    1.016327  
train loss in 01epoch  / 1900iter:    1.0127292 
train loss in 01epoc

valid loss in 01epoch  /    0iter:    0.43228686
valid loss in 01epoch  /  100iter:    0.85554443
valid loss in 01epoch  /  200iter:    0.87114752
valid loss in 01epoch  /  300iter:    0.85979364

valid 1epoch loss(349iteration):    0.85964255


# Create Submission file 

In [8]:
sample_df = pd.read_csv(input_dir + "sample_submission.csv")

In [12]:
sub_list = []
net.eval()
for img_name, img in test_generator(sample_df,WIDTH,HEIGHT):
    X = torch.tensor(img, dtype=torch.float32).to(device)
    mask_pred = net(X)
    mask_pred = mask_pred.cpu().detach().numpy()
    mask_prob = np.argmax(mask_pred, axis=1)
    mask_prob = mask_prob.ravel(order='F')
    class_dict = run_length(mask_prob,category_num)
    if len(class_dict) == 0:
        sub_list.append([img_name, "1 1", 1])
    else:
        for key, val in class_dict.items():
            sub_list.append([img_name, " ".join(map(str, val)), key])

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


In [13]:
submission_df = pd.DataFrame(sub_list, columns=sample_df.columns.values)

In [14]:
submission_df.to_csv("submission_baseline.csv", index=False)